In [9]:
!pip3 install torch torchvision torchaudio

  Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached torchvision-0.22.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.7.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-2.3.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pillow-11.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.met

In [10]:
!pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
Using cached numpy-2.2.6-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.1
    Uninstalling numpy-2.3.1:
      Successfully uninstalled numpy-2.3.1

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install albumentations

  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached scipy-1.16.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached stringzilla-3.12.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (80 kB)
  Using cached simsimd-6.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (70 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached albumentations-2.0.8-py3-none-any.whl (369 kB)
Using cached albucore-0.0.24-py3-none-any.whl (15 kB)
Using cached opencv_python_head

In [12]:
!pip install lungs-segmentation==1.1.1 --no-deps

  Using cached lungs_segmentation-1.1.1-py2.py3-none-any.whl.metadata (3.0 kB)
Using cached lungs_segmentation-1.1.1-py2.py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 이미지 경로
img_path = './data/train/0a19fb3c-36ef-401b-9deb-77f4f86d5d80.jpg'

In [27]:
#resnet34

import cv2
import numpy as np
import torch
from lungs_segmentation.inference import inference
from lungs_segmentation.pre_trained_models import create_model

# 모델 생성 및 CPU에 올림
model = create_model('resnet34').to(device='mps')
model = model.to('mps')

# 추론 실행
image, mask = inference(model, img_path)

# 결과 시각화
cv2.imwrite('masked_output_resnet34.png', mask[0] * 255)

True

In [28]:
#densenet121

import cv2
import numpy as np
import torch
from lungs_segmentation.inference import inference
from lungs_segmentation.pre_trained_models import create_model

# 모델 생성 및 CPU에 올림
model = create_model('densenet121').to(device='mps')
model = model.to('mps')

# 추론 실행
image, mask = inference(model, img_path)

# 결과 시각화
cv2.imwrite('masked_output_densenet121.png', mask[0] * 255)

True

In [26]:
mask

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], shape=(2, 512, 512), dtype=uint8)

In [ ]:
# 마스크 중 첫 번째 채널 사용 (좌/우 폐 마스크 합치기)
lung_mask = mask[0] + mask[1]

# resize to image 크기
lung_mask_resized = cv2.resize(lung_mask.astype("uint8"), (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# 마스크를 3채널로 확장
lung_mask_3ch = np.stack([lung_mask_resized]*3, axis=-1)  # shape: (H, W) → (H, W, 3)

# 곱셈: 마스크 적용
masked_image = image * lung_mask_3ch  # (H, W, 3) * (H, W, 3)

# 결과 저장
cv2.imwrite("lung_only.png", masked_image)

In [21]:
# 마스크 중 첫 번째 채널 사용 (왼쪽/오른쪽 폐 중 하나 또는 둘 합치기)
lung_mask = mask[0] + mask[1]  # 또는 np.max(mask, axis=0) 도 가능

# 원본 이미지 크기로 resize
lung_mask_resized = cv2.resize(lung_mask.astype("uint8"), (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

# 마스크 적용
masked_image = image * lung_mask_resized

# 결과 저장
cv2.imwrite("lung_only.png", masked_image)

ValueError: operands could not be broadcast together with shapes (512,512,3) (512,512) 